In [1]:
char_vocab_path = "./data/char_vocabs.txt" # 字典文件
train_data_path = "./data/train_data" # 训练数据
test_data_path = "./data/test_data" # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
label2idx = {"O": 0,
             "B-PER": 1, "I-PER": 2,
             "B-LOC": 3, "I-LOC": 4,
             "B-ORG": 5, "I-ORG": 6
             }
# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [3]:
print(train_datas[5])
print([idx2vocab[idx] for idx in train_datas[5]])
print(train_labels[5])
print([idx2label[idx] for idx in train_labels[5]])

[2158, 347, 2572, 850, 679, 5930, 2308, 6084, 563, 3765, 181, 6256, 4897, 563, 3765, 5025, 406, 3903, 915, 4110, 6802, 327, 240, 315, 2683, 352, 651, 847, 6802, 3990, 629, 3648, 341, 651, 3542, 871, 4033, 4211, 3903, 4214, 3900, 6802, 6007, 3433, 6312, 5108, 5361, 2473, 775, 181, 1208, 3007, 571, 2984, 4144, 651, 3542, 3556, 182]
['我', '们', '是', '受', '到', '郑', '振', '铎', '先', '生', '、', '阿', '英', '先', '生', '著', '作', '的', '启', '示', '，', '从', '个', '人', '条', '件', '出', '发', '，', '瞄', '准', '现', '代', '出', '版', '史', '研', '究', '的', '空', '白', '，', '重', '点', '集', '藏', '解', '放', '区', '、', '国', '民', '党', '毁', '禁', '出', '版', '物', '。']
[0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
print(tfa.__version__)

2.2.0
0.10.0


In [14]:
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K

class CRF(layers.Layer):
    def __init__(self, label_size):
        super(CRF, self).__init__()
        self.trans_params = tf.Variable(
            tf.random.uniform(shape=(label_size, label_size)), name="transition")
    
    @tf.function
    def call(self, inputs, labels, seq_lens):
        log_likelihood, self.trans_params = tfa.text.crf_log_likelihood(
                                                inputs, labels, seq_lens,
                                                transition_params=self.trans_params)
        loss = tf.reduce_sum(-log_likelihood)
        return loss

In [15]:
K.clear_session()

EPOCHS = 20
BATCH_SIZE = 64
EMBED_DIM = 128
HIDDEN_SIZE = 64
MAX_LEN = 100
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)

inputs = layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
targets = layers.Input(shape=(MAX_LEN,), name='target_ids', dtype='int32')
seq_lens = layers.Input(shape=(), name='input_lens', dtype='int32')

x = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_DIM, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(HIDDEN_SIZE, return_sequences=True))(x)
logits = layers.Dense(CLASS_NUMS)(x)
loss = CRF(label_size=CLASS_NUMS)(logits, targets, seq_lens)

model = models.Model(inputs=[inputs, targets, seq_lens], outputs=loss)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 128)     879872      input_ids[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 128)     98816       embedding[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 100, 7)       903         bidirectional[0][0]              
______________________________________________________________________________________________

In [16]:
class CustomLoss(keras.losses.Loss):
    def call(self, y_true, y_pred):
        loss, pred = y_pred
        return loss

# 自定义Loss
# model.compile(loss=CustomLoss(), optimizer='adam')
# 或者使用lambda表达式
model.compile(loss=lambda y_true, y_pred: y_pred, optimizer='adam')

In [17]:
from tensorflow.keras.preprocessing import sequence
import numpy as np

# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    datas, labels = [], []
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()
    sent_, tag_ = [], []
    for line in lines:
        if line != '\n':
            char, label = line.strip().split()
            sent_.append(char)
            tag_.append(label)
        else:
            sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
            tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
            datas.append(sent_ids)
            labels.append(tag_ids)
            sent_, tag_ = [], []
    return datas, labels

# 加载训练集
train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)

train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
train_seq_lens = np.array([MAX_LEN] * len(train_labels))
labels = np.ones(len(train_labels))
# train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)

print(np.shape(train_datas), np.shape(train_labels))

(50658, 100) (50658, 100)


In [18]:
# 训练
model.fit(x=[train_datas, train_labels, train_seq_lens], y=labels, 
        validation_split=0.1, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/20
713/713 [==============================] - 109s 152ms/step - loss: 3821.0327 - val_loss: 960.9877
Epoch 2/20
713/713 [==============================] - 117s 164ms/step - loss: 577.7767 - val_loss: 322.8379
Epoch 3/20
713/713 [==============================] - 118s 165ms/step - loss: 271.0131 - val_loss: 211.3670
Epoch 4/20
713/713 [==============================] - 114s 161ms/step - loss: 176.2213 - val_loss: 160.6102
Epoch 5/20
713/713 [==============================] - 106s 149ms/step - loss: 127.6879 - val_loss: 138.3176
Epoch 6/20
713/713 [==============================] - 106s 149ms/step - loss: 97.1733 - val_loss: 124.0059
Epoch 7/20
129/713 [====>.........................] - ETA: 1:24 - loss: 79.2221

KeyboardInterrupt: 

In [39]:
# 保存
model.save("output/bilstm_crf_ner")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: output/bilstm_crf_ner\assets


In [20]:
# model = models.load_model("output/bilstm_crf_ner", compile=False)
# 如果需要继续训练，需要下面的重新compile
# model.compile(loss=lambda y_true, y_pred: y_pred, optimizer='adam')

trans_params = model.get_layer('crf').get_weights()[0]
print(trans_params)

[[ 1.7916858  -0.90348023 -1.5236953  -1.3566005  -0.81987333 -0.98659045
  -1.1593125 ]
 [-1.1990927  -0.77374226  0.51511604 -0.11941865  0.0352908  -0.24855193
  -0.16869992]
 [-1.2247849  -0.4038762   0.4153117  -0.26688752 -0.49744508 -0.2504588
  -0.04731283]
 [-0.7922571   0.31349936 -0.7646413  -0.78899026  1.687182   -0.06874021
  -0.20729528]
 [-1.0572476   0.22198133 -0.7739668  -0.14227322  0.6633394   0.18309322
  -1.2509986 ]
 [-1.2270238  -0.27204713  0.18047248 -0.6474087  -0.3194955   0.08856065
   1.5075213 ]
 [-1.3678769  -0.64391077 -0.5975809  -0.41753736 -0.08992556 -0.4503884
   1.4988089 ]]


In [36]:
trans_params = model.get_layer('crf').get_weights()[0]
# 获得BiLSTM的输出logits
sub_model = models.Model(inputs=model.get_layer('input_ids').input,
                        outputs=model.get_layer('dense').output)

def predict(model, inputs, input_lens):
    logits = sub_model.predict(inputs)
    # 获取CRF层的转移矩阵
    # crf_decode：viterbi解码获得结果
    pred_seq, viterbi_score = tfa.text.crf_decode(logits, trans_params, input_lens)
    return pred_seq

In [37]:
maxlen = 100
sentence = "中华人民共和国国务院总理周恩来在外交部长陈毅的陪同下，连续访问了埃塞俄比亚等非洲10国以及阿尔巴尼亚。"
sent_chars = list(sentence)
sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]
sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
test_lens = np.array([100])

pred_seq = predict(model, sent2id_new, test_lens)
print(pred_seq)

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 5 6 6 6 6 6 6 6 6 6 0 0 1 2 2 0 5 6 6 0 1 2 0
  0 0 0 0 0 0 0 0 0 3 4 4 4 4 0 3 4 0 0 0 0 0 3 4 4 4 4 0]], shape=(1, 100), dtype=int32)


In [38]:
y_label = pred_seq.numpy().reshape(1, -1)[0]
print(y_label)
y_ner = [idx2label[i] for i in y_label][-len(sent_chars):]

print(sent2id)
print(y_ner)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 5 6 6 6 6 6 6 6 6 6 0 0 1 2 2 0 5 6 6 0 1 2 0 0 0
 0 0 0 0 0 0 0 3 4 4 4 4 0 3 4 0 0 0 0 0 3 4 4 4 4 0]
[242, 787, 315, 3007, 584, 966, 1208, 1208, 720, 6275, 2008, 3682, 950, 2026, 2684, 1219, 1361, 301, 5940, 6195, 6263, 2986, 3903, 6279, 889, 218, 6802, 5804, 4495, 5426, 6209, 283, 1284, 1322, 452, 2994, 296, 4277, 6362, 3148, 15, 14, 1208, 343, 843, 6256, 1639, 1778, 1654, 296, 182]
['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O']


In [31]:
# 对预测结果进行命名实体解析和提取
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            if tag_label != "O": # 当前实体tag之前有其他实体
                result_words.append((input_data[start: end], tag_label)) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                result_words.append((input_data[start: end], tag_label)) # 获取实体
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        result_words.append((input_data[start: end], tag_label)) # 获取结尾的实体
    return result_words

result_words = get_valid_nertag(sent_chars, y_ner)
for (word, tag) in result_words:
    print("".join(word), tag)

中华人民共和国国务院 ORG
周恩来 PER
外交部 ORG
陈毅 PER
埃塞俄比亚 LOC
非洲 LOC
阿尔巴尼亚 LOC
